In [1]:
# Used to access the saved FAISS vector database. 
from langchain_community.vectorstores import FAISS

# Used to embed the user's query with text-embedding-3-small 
# and send it to gpt-4o-mini.
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Used to get the statistics of each prompt, notably the input tokens,
# output tokens, and associated cost.
from langchain_community.callbacks import get_openai_callback

# Used to get the OpenAI API key from environment variables so that 
# it's not visible in this code.
import os

In [2]:
# Get the API key.
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

# I chose not to use Pinecone, as it adds considerable complication to the code.
# os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY")

# Sets the directory of the FAISS DB that's being loaded from.
# At present, it's just "FAISS", but I may make seperate DBs for different embedding models.
FAISS_PATH = "FAISS"

In [3]:
# The first step of engineering the prompt. The LLM is told to answer the question with 
# the given context. Currently, there isn't any. However, this will be formatted later
# to take this string and add the context of the relevant database chunk in place of {context}
# and add the user's query in place of {question}.
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [4]:
# Sets up the embedding model with the API key.
# Conveniently, LangChain allows for "hot-swapping" of embedding models by merely
# changing the model argument. However, using a different embedding model than the one 
# used to create the vector database will have significant negative consequences that could 
# render the chatbot inoperable, so it's essential that this matches what's used in Chroma.py.
embedder = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = os.environ["OPENAI_API_KEY"]
)

In [5]:
# Load the vector database.
db = FAISS.load_local(folder_path = FAISS_PATH, embeddings = embedder,
                      allow_dangerous_deserialization=True) # Not sure what this argument is?

In [16]:
# CURRENTLY, THIS IS THE ONLY WAY OF QUERYING THE DB AND LLM.
# I'm looking into a package called "Streamlit", which provides a web interface for user input.
# Alternatively, I think it'd be good to somehow get this as a bot on a messaging service (Teams, Discord)
# and allow queries to flow in from there. (I have yet to do any research on Teams or Discord LLM bots.)
query = "How do I make an EC claim?"

In [17]:
# Finds the three most likely chunks that the user's query applies to.
results = db.similarity_search_with_relevance_scores(query, k=3)

for doc, _score in results:
    #print(doc)
    print(_score)

# Creates the {context} previously seen in the PROMPT_TEMPLATE.
# Seperates the three most likely chunks with new lines and dashes.
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

# Formats the prompt to put the document context in place of {context},
# and the user query in place of {question}
prompt = PROMPT_TEMPLATE.format(context=context_text, question=query)
# print(prompt)

0.48894476375218254
0.2985927371375807
0.29469642958014264


In [18]:
# Used to format the LLM's output response. 
# Completely unnecessary, just makes this notebook look nicer.
from IPython.display import display, Markdown

# Create the LLM object.
## Uses GPT-4o-mini for cost efficiency.
## Temperature can be changed to vary the LLMs responses.
## Loads the API key from the system environment variables.
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
    api_key = os.environ["OPENAI_API_KEY"]
)

# To track the stats (input/output tokens, cost) of each prompt,
# get the API callback.
with get_openai_callback() as cb:
    # Gets the LLM's response.
    response = llm.invoke(prompt)

    # Attaches the document that the relevant chunks were found from.
    sources = [doc.metadata.get("source", None) for doc, _score in results]

    # Saves and outputs the LLM's response, and the sources used to generate it.
    formatted_response = f"""{llm.model_name} says:  
        {response.content}  
        Sources: {sources}"""
    display(Markdown(formatted_response))

    # Code from LangChain example (https://python.langchain.com/docs/how_to/llm_token_usage_tracking/)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion/Output Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

gpt-4o-mini says:  
        To make an Extenuating Circumstances (EC) claim, follow these steps:

1. Log in to your profile on MySRS at mysrs.bcu.ac.uk.
2. Click on the ‘Support’ tab.
3. Select the ‘Extenuating Circumstances’ option.
4. Ensure to include all assessments you wish to claim for in your submission.
5. If you encounter any technical issues, contact mysrs@bcu.ac.uk for assistance.
6. If you do not have access to MySRS or cannot make a claim electronically, reach out to Student Governance at appealsandresolutions@bcu.ac.uk for an alternative method to submit your claim.

Additionally, refer to the separate User Guidance document available from iCity for step-by-step instructions on making a claim through MySRS.  
        Sources: ['Data\\Policies\\ExtenuatingCircumstances.pdf', 'Data\\Policies\\AssessmentAndFeedback.pdf', 'Data\\Policies\\ExtenuatingCircumstances.pdf']

Total Tokens: 753
Prompt Tokens: 585
Completion/Output Tokens: 168
Total Cost (USD): $0.00018855
